In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import matplotlib.pyplot as plt
import  networkx as nx
import tensorflow as tf

%matplotlib inline

In [2]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A


communities = 2 #number of communities, chance to 
group_size = 6 #number of nodes in each communitites (balanced so far)
dim_graph = communities*group_size
A = np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.double)

data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.float64) for i in range(100)]
data[1]

array([[ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.]])

In [3]:
communities = 6
Adj = tf.placeholder(tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float64),
                                      tf.ones([communities], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float64),
                                      tf.zeros([communities], dtype=tf.float64)])


y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])

centers = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(centers, 1)

distances = tf.transpose(tf.reduce_sum(tf.square(tf.sub(expanded_vectors,
                                                        expanded_centers)), 2))

assignments = tf.nn.softmax(distances)

loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(tf.reduce_sum(loss_vec_a), tf.reduce_sum(loss_vec_a))



optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, centers])

with tf.Session() as sess:
    for i in range(10):
        sess.run(init)
#       sess.run(train)
        print sess.run([r, loss, centers], feed_dict = {Adj: data[i]})
    
    





[array([ 3.05923993]), -17.582002877105602, array([[ 0.48523878, -0.67196621],
       [-0.98420493,  1.006611  ]])]
[array([-1.99537013]), -14.645440402921187, array([[ 0.70392115, -1.39593814],
       [ 0.351965  ,  0.15934262]])]
[array([-0.92830202]), -8.6270355575422322, array([[ 1.87739813, -0.60799193],
       [ 1.78306146, -0.52507174]])]
[array([-0.65208576]), -14.781249747021276, array([[ 0.65875418, -1.1032957 ],
       [ 0.83589625,  1.45952181]])]
[array([ 0.51431394]), -6.4003825869010535, array([[-0.06595127, -0.43541562],
       [ 0.28237509,  0.78109758]])]
[array([-0.15450143]), -14.609192468897993, array([[-0.31750946,  0.83904036],
       [-0.87075515,  1.5013767 ]])]
[array([-1.55449834]), -28.915727487177978, array([[-0.70682717,  2.27252048],
       [-0.36664564,  0.84933227]])]
[array([-0.25043337]), -39.765463118162693, array([[ 1.37487904,  3.01357896],
       [-2.21581987, -0.49357137]])]
[array([ 0.21729751]), -25.181655264180588, array([[-0.52254283, -1.0943

In [4]:
#now with training uncommentedout

In [39]:
communities = 6
Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float64),
                                      tf.ones([communities], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float64),
                                      tf.zeros([communities], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 

eigenvec_proj = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))
eigenvec_proj = tf.cast(tf.reduce_sum(eigenvec, 1), dtype = tf.float64)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))


optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r])

with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run(train, feed_dict = {Adj: data[i]})
        print sess.run([r, loss_grad, eigenvec_grad, loss, y_a], feed_dict = {Adj: data[i]})
    
    



[array([-0.80005489]), [array([ 9.79694719])], [array([-1.98701186])], 80.859714974926163, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.43659751]), [array([-4.58307354])], [array([ 1.68392781])], 77.553692330113861, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.4539848]), [array([ 0.31788765])], [array([-1.11917682])], 81.061766717813185, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([ 0.12698334]), [array([ 11.42162481])], [array([-8.62615963])], 79.439033997694281, array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])]
[array([-0.3181855]), [array([ 6.73951226])], [array([ 5.23449881])], 82.768

In [33]:

tf.reset_default_graph()

In [38]:
?tf.pack


In [8]:
tw = tf.train.SummaryWriter('./logDir',sess.graph)

# Train your network

tw.flush()
tw.close()

In [38]:
communities = 2 #number of communities, chance to 
group_size = 5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.8, p_out=0.3)).astype(np.float64) for i in range(1000)]
data[1]

array([[ 0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.]])

In [53]:
communities = 2
group_size=5
dim_graph=communities*group_size

Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

average_degree = tf.reduce_sum(Adj)/dim_graph

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


center = tf.Variable(tf.truncated_normal(shape=[2,2],
                                       mean=0.0, stddev=0.6, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(center, 1)

distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

hard_assignment = tf.argmin(distances, 0)
new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

#a = tf.slice(new_centers, [0,0], [1,2])
#b = tf.slice(new_centers, [1,0], [1,2])
update_centers = tf.assign(center, new_centers)


assignments = 1-tf.nn.softmax(tf.transpose(distances)) #this ensures that the smaller distances are actually the correct assignments as being far from the centers make it less likely it is in that cluster
#warning, the above transform will definitely not work for more than 2 centers, so need to be more clever later...
#like basically sum the other two entries and subtract, this will be the generalized algo.... 


loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_b))


#loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))),tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))


optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, center])

with tf.Session() as sess:
    sess.run(init)
    sess.run(train, feed_dict = {Adj: data[i]})
    for i in range(100):
        sess.run([train], feed_dict = {Adj: data[i]})
        if i%10 ==0:
            print sess.run([loss, hard_assignment, r, average_degree, center, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



[5.786448819254435, array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1]), array([ 0.95610812]), 4.2000000000000002, array([[-0.15236526,  0.81405732],
       [ 0.14181189, -0.15286577]]), array([[-0.31730845,  0.32506619],
       [-0.31588982, -0.07618329]])]
[5.1170808988547529, array([1, 1, 1, 1, 1, 0, 0, 0, 1, 1]), array([ 1.64406844]), 5.2000000000000002, array([[-0.15236526,  0.81405732],
       [ 0.14181189, -0.15286577]]), array([[-0.28199102,  0.41914946],
       [-0.3288483 , -0.15155647]])]
[4.7725667909998837, array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0]), array([ 2.32061393]), 5.7999999999999998, array([[-0.15236526,  0.81405732],
       [ 0.14181189, -0.15286577]]), array([[-0.32764608,  0.31990748],
       [-0.31036049, -0.15957133]])]
[5.3100322273663849, array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([ 2.55736504]), 4.5999999999999996, array([[-0.15236526,  0.81405732],
       [ 0.14181189, -0.15286577]]), array([[        nan,         nan],
       [ 0.31444076, -0.01014541]])]
[4.74973777310695

In [ ]:

tf.reset_default_graph()

In [45]:
#does backprop not play well with update centers? 
?tf.assign


In [119]:


center_1 = tf.Variable(tf.random_normal(shape=[2,1],
                                       mean=1.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))
center_2 = tf.Variable(tf.random_normal(shape=[2,1],
                                       mean=-1.0, stddev=1.0, dtype=tf.float64,seed=None, name=None))
center = tf.pack([center_1, center_2])

In [115]:
communities = 2
group_size=10
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.1)).astype(np.float64) for i in range(1000)]




dim_graph=communities*group_size

Adj = tf.placeholder(tf.float64)
Adj = tf.cast(Adj, tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)

#r =  tf.Variable(tf.constant(2.1), dtype = tf.float64)
r = tf.Variable(tf.truncated_normal(shape=[1], mean=2.0, stddev=0.05, dtype=tf.float64,seed=None, name=None))

Bethe_Hesse_neg = (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

y_a = tf.pack([true_assignment_a, true_assignment_b]) 
y_b = tf.pack([true_assignment_b, true_assignment_a]) 


eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


center = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=2.0, dtype=tf.float64,seed=None, name=None))

expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
expanded_centers = tf.expand_dims(center, 1)

distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

hard_assignment = tf.argmin(distances, 0)
new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

#a = tf.slice(new_centers, [0,0], [1,2])
#b = tf.slice(new_centers, [1,0], [1,2])
a = tf.shape(center)
b = tf.shape(new_centers)
update_centers = tf.assign(center, new_centers)


assignments = 1-tf.nn.softmax(tf.transpose(distances)) #this ensures that the smaller distances are actually the correct assignments as being far from the centers make it less likely it is in that cluster
#warning, the above transform will definitely not work for more than 2 centers, so need to be more clever later...
#like basically sum the other two entries and subtract, this will be the generalized algo.... 


loss_vec_a = y_a*tf.log(tf.transpose(assignments))
loss_vec_b = y_b*tf.log(tf.transpose(assignments))

loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_b))


#loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))),tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))


optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss, var_list=[r])

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r, center])

init_center = tf.initialize_variables([center])

with tf.Session() as sess:
    sess.run(init)
    for i in range(100):
        sess.run([train], feed_dict = {Adj: data[i]})
        if i%1 ==0:
            print i
            print sess.run([update_centers, loss, hard_assignment, r, average_degree_root, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



0
[array([[-0.22750467, -0.0530663 ],
       [-0.19108906,  0.30987358]]), 10.543278643139828, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]), array([ 2.06279911]), 2.4494897427831779, array([[-0.22750467, -0.0530663 ],
       [-0.19108906,  0.30987358]])]
1
[array([[-0.21549371, -0.18718428],
       [-0.22546771,  0.22463048]]), 12.444116752146991, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]), array([ 2.05917424]), 2.1908902300206643, array([[-0.21549371, -0.18718428],
       [-0.22546771,  0.22463048]])]
2
[array([[ 0.21542102, -0.22239284],
       [ 0.22719654,  0.2105021 ]]), 12.167981712621611, array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ 2.05930951]), 2.4289915602982237, array([[ 0.21542102, -0.22239284],
       [ 0.22719654,  0.2105021 ]])]
3
[array([[-0.20751116, -0.19755593],
       [-0.23031046,  0.20657325]]), 12.333687791610281, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]), ar

In [27]:

tf.reset_default_graph()

In [133]:
communities = 2
group_size=10
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.1)).astype(np.float64) for i in range(1000)]


def Bethe_Hessian_NN(communities, group_size, data):

    dim_graph=communities*group_size

    Adj = tf.placeholder(tf.float64)
    Adj = tf.cast(Adj, tf.float64)

    Diag = tf.diag(tf.reduce_sum(Adj,0))
    Diag = tf.cast(Diag, tf.float64)

    average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)


    r = tf.Variable(tf.truncated_normal(shape=[1], mean=2.0, stddev=0.05, dtype=tf.float64,seed=None, name=None))

    Bethe_Hesse_neg = (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

    eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

    true_assignment_a = tf.concat(0, [tf.zeros([group_size], dtype=tf.float64),
                                      tf.ones([group_size], dtype=tf.float64)])
    true_assignment_b = tf.concat(0, [tf.ones([group_size], dtype=tf.float64),
                                      tf.zeros([group_size], dtype=tf.float64)])

    y_a = tf.pack([true_assignment_a, true_assignment_b]) 
    y_b = tf.pack([true_assignment_b, true_assignment_a]) 


    eigenvec_proj = tf.slice(eigenvec, [0,0], [dim_graph, 2])


    center = tf.Variable(tf.random_normal(shape=[2,2],
                                       mean=0.0, stddev=3.0, dtype=tf.float64,seed=None, name=None))

    expanded_vectors = tf.expand_dims(eigenvec_proj, 0)
    expanded_centers = tf.expand_dims(center, 1)

    distances = tf.reduce_sum(tf.square(tf.sub(expanded_vectors, expanded_centers)), 2)

    hard_assignment = tf.argmin(distances, 0)
    new_centers = tf.concat(0, [tf.reduce_mean(tf.gather(eigenvec_proj,
                                                     tf.reshape(tf.where(tf.equal(hard_assignment, c)),
                                                                [1,-1])), reduction_indices=[1]) for c in xrange(communities)])

    update_centers = tf.assign(center, new_centers)


    assignments = 1-tf.nn.softmax(tf.transpose(distances)) 

    loss_vec_a = y_a*tf.log(tf.transpose(assignments))
    loss_vec_b = y_b*tf.log(tf.transpose(assignments))

    loss = tf.minimum(-tf.reduce_sum(loss_vec_a), -tf.reduce_sum(loss_vec_b))


    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = optimizer.minimize(loss, var_list=[r])

    eigenvec_grad = tf.gradients(eigenvec, r)
    loss_grad = tf.gradients(loss, r)

    init = tf.initialize_variables([r, center])

    init_center = tf.initialize_variables([center])

    with tf.Session() as sess:
        sess.run(init)
        for i in range(100):
            sess.run([train], feed_dict = {Adj: data[i]})
            if i%3==0:
                print i
                print sess.run(center, feed_dict={Adj: data[i]})
            if i%2 ==0:
                print i
                print sess.run([update_centers, loss, hard_assignment, r, average_degree_root, new_centers], feed_dict = {Adj: data[i]})
    
#, distances, assignments,  loss_grad, eigenvec_grad, 



In [134]:
Bethe_Hessian_NN(2, 10, data)

0
[[-1.66639986  1.61819571]
 [-0.76720706  3.83602632]]
0
[array([[-0.21554302,  0.02405878],
       [        nan,         nan]]), 92.742087970598988, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ 2.3522458]), 2.2583179581272428, array([[-0.21554302,  0.02405878],
       [        nan,         nan]])]


InvalidArgumentError: Self Adjoint Eigen decomposition was not successful. The input might not be valid.
	 [[Node: SelfAdjointEigV2_91 = SelfAdjointEigV2[T=DT_DOUBLE, compute_v=true, _device="/job:localhost/replica:0/task:0/cpu:0"](add_91)]]
Caused by op u'SelfAdjointEigV2_91', defined at:
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packa

In [245]:

def EigenDecomp_r_diff(iterations, communities, group_size, data, r1, r2):

    dim_graph=communities*group_size

    Adj = tf.placeholder(tf.float64)
    Adj = tf.cast(Adj, tf.float64)

    Diag = tf.diag(tf.reduce_sum(Adj,0))
    Diag = tf.cast(Diag, tf.float64)

    average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)
    
    r1 = tf.cast(r1, dtype=tf.float64)
    r2 = tf.cast(r2, dtype=tf.float64)
    
    def BH(r):
        return (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 



    BH1 = BH(r1)
    BH2 = BH(r2)

    eigenval1, eigenvec1 = tf.self_adjoint_eig(BH1)
    eigenval2, eigenvec2 = tf.self_adjoint_eig(BH2)
    
    e1vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec1, eigenvec1)))
    e2vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec2, eigenvec2)))
    
    operator_diff = tf.reduce_sum(tf.matmul(eigenvec1, eigenvec2))/(e1vec_norm*e2vec_norm)
    
    e1_norm = tf.sqrt(tf.reduce_sum(eigenval1*eigenval1))
    e2_norm = tf.sqrt(tf.reduce_sum(eigenval2*eigenval2))
    eigenval_dot = tf.reduce_sum(eigenval1*eigenval2)/(e1_norm*e2_norm)

    with tf.Session() as sess:
        for i in range(iterations):
            print i
            print sess.run([average_degree_root, eigenval_dot, operator_diff], feed_dict = {Adj: data[i]})


def EigenDecomp_r_diff_proj(iterations, communities, group_size, data, r1, r2, project_dim):

    dim_graph=communities*group_size

    Adj = tf.placeholder(tf.float64)
    Adj = tf.cast(Adj, tf.float64)

    Diag = tf.diag(tf.reduce_sum(Adj,0))
    Diag = tf.cast(Diag, tf.float64)

    average_degree_root = tf.sqrt(tf.reduce_sum(Adj)/dim_graph)
    
    r1 = tf.cast(r1, dtype=tf.float64)
    r2 = tf.cast(r2, dtype=tf.float64)
    
    def BH(r):
        return (r*r-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 


    BH1 = BH(r1)
    BH2 = BH(r2)

    eigenval1, eigenvec1 = tf.self_adjoint_eig(BH1)
    eigenval2, eigenvec2 = tf.self_adjoint_eig(BH2)
    
    eigenvec1 = tf.slice(eigenvec1, [0,0], [dim_graph, project_dim])
    eigenvec2 = tf.slice(eigenvec2, [0,0], [dim_graph, project_dim])
    
    e1vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec1, tf.transpose(eigenvec1))))
    e2vec_norm = tf.sqrt(tf.reduce_sum(tf.matmul(eigenvec2, tf.transpose(eigenvec2))))
    
    operator_diff = tf.reduce_sum(tf.matmul(eigenvec1, tf.transpose(eigenvec2)))/(e1vec_norm*e2vec_norm)
    
    e1_norm = tf.sqrt(tf.reduce_sum(eigenval1*eigenval1))
    e2_norm = tf.sqrt(tf.reduce_sum(eigenval2*eigenval2))
    eigenval_dot = tf.reduce_sum(eigenval1*eigenval2)/(e1_norm*e2_norm)

    lst = []
    
    with tf.Session() as sess:
        for i in range(iterations):
            
            lst.append(sess.run([average_degree_root, eigenval_dot, operator_diff], feed_dict = {Adj: data[i]}))
    return np.asarray(lst)
            
  


In [221]:
communities = 2
group_size=5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(np.float64) for i in range(1000)]


EigenDecomp_r_diff(100, 2, 5, data, 4, 4.5)

0
[1.3416407864998738, 0.99950496573195446, nan]
1
[1.8973665961010275, 0.9995177687259964, 0.13649340617465697]
2
[1.4142135623730951, 0.99947153734202154, 0.16781677434272257]
3
[1.61245154965971, 0.99950109600895498, 0.98817899950159394]
4
[2.0976176963403033, 0.99956104225878661, -1.0718698711283243]
5
[1.7320508075688772, 0.99950507875216699, nan]
6
[1.5491933384829668, 0.99940355715928153, 1.3619421046282456]
7
[1.8439088914585775, 0.99952465280469971, 0.99584888351341805]
8
[1.8439088914585775, 0.999505363097972, nan]
9
[1.7888543819998317, 0.99950626465120362, 1.0354907929844397]
10
[1.7320508075688772, 0.99951686233267822, 2.535177781630642]
11
[1.9493588689617927, 0.99949034909303747, 1.0152560966467397]
12
[1.6733200530681511, 0.99947668796809253, 0.998021069428361]
13
[1.61245154965971, 0.99943534894450914, -2.7894082447146489]
14
[1.7888543819998317, 0.99950651837122184, nan]
15
[1.7320508075688772, 0.99950354426611931, nan]
16
[1.7888543819998317, 0.99948797990356342, nan

In [181]:

tf.reset_default_graph()

In [214]:
communities = 2
group_size=5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=1.0, p_out=0.0)).astype(np.float64) for i in range(1000)]


EigenDecomp_r_diff(10, 2, 5, data, 2, 3)

0
[2.0, 0.99846035320541238, nan]
1
[2.0, 0.99846035320541238, nan]
2
[2.0, 0.99846035320541238, nan]
3
[2.0, 0.99846035320541238, nan]
4
[2.0, 0.99846035320541238, nan]
5
[2.0, 0.99846035320541238, nan]
6
[2.0, 0.99846035320541238, nan]
7
[2.0, 0.99846035320541238, nan]
8
[2.0, 0.99846035320541238, nan]
9
[2.0, 0.99846035320541238, nan]


In [260]:
communities = 2
group_size=5
data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.15, p_out=0.003)).astype(np.float64) for i in range(1000)]

answer = EigenDecomp_r_diff_proj(100, 2, 5, data, 1, 7, 10)

In [261]:
answer[:,2]
np.mean(np.abs(answer[:,2]))

0.88547375800806716